In [1]:
import os
import warnings
import sys

import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.sklearn

In [2]:
data_path = "wine-quality.csv"
data = pd.read_csv(data_path)

data.sample(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
4364,6.4,0.290,0.21,9.65,0.041,36.0,119.0,0.99334,2.99,0.34,10.933333,6
1586,9.3,0.310,0.49,1.30,0.042,34.0,147.0,0.99480,3.11,0.46,9.800000,5
4792,4.7,0.785,0.00,3.40,0.036,23.0,134.0,0.98981,3.53,0.92,13.800000,6
3482,5.4,0.450,0.27,6.40,0.033,20.0,102.0,0.98944,3.22,0.27,13.400000,8
3281,6.5,0.320,0.30,2.30,0.051,20.0,127.0,0.98964,3.13,0.52,12.800000,6
840,7.6,0.330,0.35,6.30,0.036,12.0,126.0,0.99240,3.16,0.39,12.000000,7
3005,6.9,0.300,0.36,0.90,0.037,40.0,156.0,0.98968,3.08,0.36,12.100000,6
328,6.5,0.280,0.35,15.40,0.042,55.0,195.0,0.99780,3.23,0.50,9.600000,6
4256,6.0,0.330,0.26,5.10,0.051,16.0,119.0,0.99416,3.15,0.41,9.200000,5
4367,7.1,0.310,0.25,11.20,0.048,32.0,136.0,0.99663,3.14,0.40,9.500000,5


In [3]:
# mlflow server --backend-store-uri mlruns/ --default-artifact-root mlruns/ --host localhost --port 5006


remote_server_uri = "http://localhost:5006" # set to your server URI
mlflow.set_tracking_uri(remote_server_uri)  # or set the MLFLOW_TRACKING_URI in the env
mlflow.tracking.get_tracking_uri()

'http://localhost:5006'

In [4]:
exp_name = "ElasticNet_wine"
mlflow.set_experiment(exp_name)

<Experiment: artifact_location='/home/carolyn/mlflow_assignment/mlruns/274904185405977709', creation_time=1723029212142, experiment_id='274904185405977709', last_update_time=1723029212142, lifecycle_stage='active', name='ElasticNet_wine', tags={}>

In [5]:
def eval_metrics(actual, pred):
    # compute relevant metrics
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2


def load_data(data_path):
    data = pd.read_csv(data_path)

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]
    return train_x, train_y, test_x, test_y


def train(alpha=0.5, l1_ratio=0.5):
    # train a model with given parameters
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read the wine-quality csv file (make sure you're running this from the root of MLflow!)
    data_path = "wine-quality.csv"
    train_x, train_y, test_x, test_y = load_data(data_path)

    # Useful for multiple runs (only doing one run in this sample notebook)    
    with mlflow.start_run():
        # Execute ElasticNet
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)
        # Evaluate Metrics
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        # Print out metrics
        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log parameter, metrics, and model to MLflow
        mlflow.log_param(key="alpha", value=alpha)
        mlflow.log_param(key="l1_ratio", value=l1_ratio)
        mlflow.log_metric(key="rmse", value=rmse)
        mlflow.log_metrics({"mae": mae, "r2": r2})
        mlflow.log_artifact(data_path)
        print("Save to: {}".format(mlflow.get_artifact_uri()))
        
        mlflow.sklearn.log_model(lr, "model")


In [6]:
train(0.5, 0.5)

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.82224284975954
  MAE: 0.6278761410160693
  R2: 0.12678721972772689
Save to: /home/carolyn/mlflow_assignment/mlruns/274904185405977709/96580a2c6e5e455c8299e9db779b96fa/artifacts


2024/08/08 12:06:55 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/08 12:06:55 INFO mlflow.tracking._tracking_service.client: 🏃 View run secretive-goose-187 at: http://localhost:5006/#/experiments/274904185405977709/runs/96580a2c6e5e455c8299e9db779b96fa.
2024/08/08 12:06:55 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5006/#/experiments/274904185405977709.


In [7]:
train(0.2, 0.2)

Elasticnet model (alpha=0.200000, l1_ratio=0.200000):
  RMSE: 0.7859129997062341
  MAE: 0.6155290394093893
  R2: 0.20224631822892103
Save to: /home/carolyn/mlflow_assignment/mlruns/274904185405977709/a5d81cb4d6da443e83374a4b7027a072/artifacts


2024/08/08 12:06:57 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/08 12:06:57 INFO mlflow.tracking._tracking_service.client: 🏃 View run wise-goose-265 at: http://localhost:5006/#/experiments/274904185405977709/runs/a5d81cb4d6da443e83374a4b7027a072.
2024/08/08 12:06:57 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5006/#/experiments/274904185405977709.


In [8]:
train(0.1, 0.1)

Elasticnet model (alpha=0.100000, l1_ratio=0.100000):
  RMSE: 0.7792546522251949
  MAE: 0.6112547988118587
  R2: 0.2157063843066196
Save to: /home/carolyn/mlflow_assignment/mlruns/274904185405977709/77f0a4a1eb124e11946a554e54402019/artifacts


2024/08/08 12:07:00 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/08 12:07:00 INFO mlflow.tracking._tracking_service.client: 🏃 View run languid-skunk-600 at: http://localhost:5006/#/experiments/274904185405977709/runs/77f0a4a1eb124e11946a554e54402019.
2024/08/08 12:07:00 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5006/#/experiments/274904185405977709.


In [9]:
print(mlflow.__version__)

2.15.1


In [10]:
# pip install --upgrade mlflow


In [11]:
from mlflow.tracking import MlflowClient

# Create an instance of MlflowClient
client = MlflowClient()

# List all experiments
experiments = client.search_experiments()

# Print experiment details
for experiment in experiments:
    print(f"ID: {experiment.experiment_id}, Name: {experiment.name}, Artifact Location: {experiment.artifact_location}, Lifecycle Stage: {experiment.lifecycle_stage}")


ID: 274904185405977709, Name: ElasticNet_wine, Artifact Location: /home/carolyn/mlflow_assignment/mlruns/274904185405977709, Lifecycle Stage: active
ID: 0, Name: Default, Artifact Location: /home/carolyn/mlflow_assignment/mlruns/0, Lifecycle Stage: active


In [12]:
# get the run
_run = client.get_run(run_id="b668b7821b1f4485a1d8100c1234bab1")
print(_run)

<Run: data=<RunData: metrics={'mae': 0.6112547988118587,
 'r2': 0.2157063843066196,
 'rmse': 0.7792546522251949}, params={'alpha': '0.1', 'l1_ratio': '0.1'}, tags={'deployed': '07-08-2024 (19:43:23.476683)',
 'mlflow.log-model.history': '[{"run_id": "b668b7821b1f4485a1d8100c1234bab1", '
                             '"artifact_path": "model", "utc_time_created": '
                             '"2024-08-07 14:09:24.036450", "flavors": '
                             '{"python_function": {"model_path": "model.pkl", '
                             '"predict_fn": "predict", "loader_module": '
                             '"mlflow.sklearn", "python_version": "3.10.12", '
                             '"env": {"conda": "conda.yaml", "virtualenv": '
                             '"python_env.yaml"}}, "sklearn": '
                             '{"pickled_model": "model.pkl", '
                             '"sklearn_version": "1.5.1", '
                             '"serialization_format": "cloudpick

In [13]:
# add a tag to the run
from datetime import datetime

dt = datetime.now().strftime("%d-%m-%Y (%H:%M:%S.%f)")
client.set_tag(_run.info.run_id, "deployed", dt)